### Prototype data Precessing with Spacy

1. Import model
2. Import database
3. iterate through database and label the data
4. Create DF with labeled data as features
5. Export DF as 'results.csv'

In [1]:
import spacy
import pandas as pd 
import numpy as np 

In [2]:
# Loading the Model

nlp = spacy.load("../training/test_model")

In [3]:
# Loading the input_data

data = pd.read_csv("input_data_filtered.csv")

In [11]:
data["PrimaryOutcomeMeasure"] = data["PrimaryOutcomeMeasure"].astype(str)
data["PrimaryTimeFrame"] = data["PrimaryTimeFrame"].astype(str)
data["SecondaryOutcomeMeasure"] = data["SecondaryOutcomeMeasure"].astype(str)
data["SecondaryTimeFrame"] = data["SecondaryTimeFrame"].astype(str)

In [25]:
data_result_df = pd.DataFrame(columns=["NCTId", "Condition", 'Change', "Reference", "Variable","Timepoint"])

In [26]:
data_result_df

,NCTId,Condition,Change,Reference,Variable,Timepoint


In [47]:
# Iteration through data 
# TODO: Remove Measurment-Title ???
# TODO: Process for multi-row data


dictTest = {"NCTId":[],"Condition":[],"Change":[],"Reference":[], "Variable" :[], "Timepoint":[]}
for index, row in data.iterrows():

    trial_id = row[1]
    dictTest["NCTId"].append(trial_id)
    print(data_result_df["NCTId"])

    condition = row[2]
    dictTest["Condition"].append(condition)

    primary_endpoint = row[3]
    primary_endpoints = []

    primary_endpoint_timepoint = row[4]
    primary_endpoint_timepoints = []

    secondary_endpoint = row[5]
    secondary_endpoints = []

    secondary_endpoint_timepoint = row[6]
    secondary_endpoint_timepoints = []

    if  primary_endpoint.find("|") == 1:
        primary_endpoints = primary_endpoint.split("|")
        print(primary_endpoints)

    if  primary_endpoint_timepoint.find("|") == 1:
        primary_endpoint_timepoints = primary_endpoint_timepoint.split("|")
        print(primary_endpoint_timepoint)


    if  secondary_endpoint.find("|") == 1:
        secondary_endpoints = secondary_endpoint.split('|')
        print(secondary_endpoint)

    if  secondary_endpoint_timepoint.find("|") == 1:
        secondary_endpoint_timepoints = secondary_endpoint_timepoint.split('|')
        print(secondary_endpoint_timepoint)


    target = nlp(primary_endpoint)
    for entity in target.ents:
        dictTest[entity.label_].append(entity.text)


    target = nlp(secondary_endpoint)

    for entity in target.ents:
        dictTest[entity.label_].append(entity.text)


Series([], Name: NCTId, dtype: object)
{'NCTId': ['NCT01814787'], 'Condition': ['Type 2 Diabetes', 'Type 2 Diabetes'], 'Change': ['Number of Children'], 'Reference': [], 'Variable': ['Documented Risk Factors'], 'Timepoint': []}
Series([], Name: NCTId, dtype: object)
{'NCTId': ['NCT01814787', 'NCT01346033'], 'Condition': ['Type 2 Diabetes', 'Type 2 Diabetes', 'Type 2 Diabetes', 'type 2 diabetes'], 'Change': ['Number of Children'], 'Reference': ['Validation'], 'Variable': ['Documented Risk Factors', 'SCOUT DS algorithm'], 'Timepoint': []}
Series([], Name: NCTId, dtype: object)
{'NCTId': ['NCT01814787', 'NCT01346033', 'NCT04765631'], 'Condition': ['Type 2 Diabetes', 'Type 2 Diabetes', 'Type 2 Diabetes', 'type 2 diabetes', 'Type 2 Diabetes', 'type 2 diabetics'], 'Change': ['Number of Children', 'outcome'], 'Reference': ['Validation', 'comparison'], 'Variable': ['Documented Risk Factors', 'SCOUT DS algorithm', 'total volumetric bone mineral density (vBMD)'], 'Timepoint': []}
Series([], Name

In [54]:
dictTest

{'NCTId': ['NCT01814787',
  'NCT01346033',
  'NCT04765631',
  'NCT03979768',
  'NCT02286128',
  'NCT03362762',
  'NCT01920256',
  'NCT03840850',
  'NCT00392678',
  'NCT03159221',
  'NCT00485758',
  'NCT03044860',
  'NCT02355145',
  'NCT01257087',
  'NCT03130894',
  'NCT02440555',
  'NCT02327429',
  'NCT03259321',
  'NCT04016584',
  'NCT00810823',
  'NCT01601574',
  'NCT03243136',
  'NCT01021865',
  'NCT01055652',
  'NCT02290860',
  'NCT04005261',
  'NCT00612794',
  'NCT01727349',
  'NCT00239187',
  'NCT01073020',
  'NCT01249677',
  'NCT01085292',
  'NCT01182935',
  'NCT02205996',
  'NCT00240253',
  'NCT00982371',
  'NCT04031417',
  'NCT01284465',
  'NCT02226822',
  'NCT01869608',
  'NCT01184768',
  'NCT04501991',
  'NCT01010035',
  'NCT01387984',
  'NCT01237301',
  'NCT04403841',
  'NCT01088711',
  'NCT03214380',
  'NCT02322762',
  'NCT00888836',
  'NCT03447275',
  'NCT00451620',
  'NCT01845064',
  'NCT02104804',
  'NCT00787670',
  'NCT01933256',
  'NCT03406910',
  'NCT01649466',
  'NC

In [49]:
results = data_result_df.from_dict(data)

In [55]:
results.head()

,Unnamed: 0,NCTId,Condition,PrimaryOutcomeMeasure,PrimaryTimeFrame,SecondaryOutcomeMeasure,SecondaryTimeFrame,BaselineMeasureTitle
0,1,NCT01814787,Type 2 Diabetes,Number of Children With Documented Risk Factor...,12 months,nan,nan,"Age, Categorical|Sex: Female, Male|Race/Ethnic..."
1,2,NCT01346033,Type 2 Diabetes,Validation of SCOUT DS algorithm for detecting...,1 day,nan,nan,NaN
2,3,NCT04765631,Type 2 Diabetes,The main outcome is the comparison of total vo...,at inclusion for Qualyor patients. (the HRpQCT...,nan,nan,NaN
3,4,NCT03979768,Type2 Diabetes,Number of Participants with undiagnosed type 2...,2 months,Explore if there was any difference in the num...,Baseline,NaN
4,5,NCT02286128,Type 2 Diabetes,Correlation between NF-кB dependent-proinflamm...,2-4 weeks,Correlation between NF-кB dependent-proinflamm...,2-4 weeks|2-4 weeks,NaN
